In [1]:
import os

In [2]:
!nvidia-smi

Sun Jan 26 14:16:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    On  |   00000000:01:00.0 Off |                    0 |
| 30%   42C    P8             29W /  300W |    2647MiB /  46068MiB |     40%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# /home/integration/test/new

In [4]:
os.getcwd()

'/home/integration/test/new'

In [5]:
os.listdir()

['florence_2_250M_from_cloned_repo.ipynb',
 'samplehololensimage.jpg',
 'florence_2_from_cloned_repo.ipynb',
 'old_saved_model_checkpoints',
 'Florence-2-base-ft',
 'Florence-2-large-ft']

In [6]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

# Path to your cloned repository
small_model_local_model_path = "./Florence-2-base-ft"  # or full path like "/path/to/Florence-2-large-ft"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load model from local path
small_model = AutoModelForCausalLM.from_pretrained(
    small_model_local_model_path,
    torch_dtype=torch_dtype,
    trust_remote_code=True
).to(device)

# Load processor from local path
small_processor = AutoProcessor.from_pretrained(
    small_model_local_model_path,
    trust_remote_code=True
)

/opt/conda/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise y

In [7]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

# Path to your cloned repository
large_model_local_model_path = "./Florence-2-large-ft"  # or full path like "/path/to/Florence-2-large-ft"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load model from local path
large_model = AutoModelForCausalLM.from_pretrained(
    large_model_local_model_path,
    torch_dtype=torch_dtype,
    trust_remote_code=True
).to(device)

# Load processor from local path
large_processor = AutoProcessor.from_pretrained(
    large_model_local_model_path,
    trust_remote_code=True
)

In [8]:
def run_example(model, processor, image, task_prompt, text_input=None ,tensor_checkpoint_id=""):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    
    model.forward_pass_tensor_checkpoint_id = tensor_checkpoint_id
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text, 
        task=task_prompt, 
        image_size=(image.width, image.height)
    )


    return parsed_answer


In [10]:
hololens_image = Image.open('samplehololensimage.jpg')

In [ ]:
task_prompt = '<DETAILED_CAPTION>'
run_example(model=large_model, processor=large_processor, image=hololens_image, task_prompt=task_prompt, text_input=None)

In [ ]:
task_prompt = '<DETAILED_CAPTION>'
run_example(model=small_model, processor=small_processor, image=hololens_image, task_prompt=task_prompt, text_input=None)

In [ ]:
!nvidia-smi

In [11]:
task_prompt = ''
run_example(model=small_model, processor=small_processor, image=hololens_image, task_prompt=task_prompt, text_input="Where is the fan?")

Florence-2: vision 50% attention map saved
Florence-2: text_encoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved


{'': 'on desk'}

In [13]:
task_prompt = ''
run_example(model=large_model, processor=large_processor, image=hololens_image, task_prompt=task_prompt, text_input="Where is the fan?")

Florence-2: vision 50% attention map saved
Florence-2: text_encoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved


{'': 'on shelf'}

In [14]:
task_prompt = ''
run_example(model=small_model, processor=small_processor, image=hololens_image, task_prompt=task_prompt, text_input="Where is the trash can?")

Florence-2: vision 50% attention map saved
Florence-2: text_encoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved
Florence-2 (base): text_decoder 50% attention map saved


{'': 'on floor'}

In [15]:
task_prompt = ''
run_example(model=large_model, processor=large_processor, image=hololens_image, task_prompt=task_prompt, text_input="Where is the trash can?")

Florence-2: vision 50% attention map saved
Florence-2: text_encoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved
Florence-2: text_decoder 50% attention map saved


{'': 'floor'}

In [ ]:
# forward pass for storing intermediate attention maps


In [ ]:
# Load the tensor 
os.listdir()

In [ ]:
tensor1.shape

In [ ]:
model1 = 

In [ ]:
model2 = 

In [ ]:
model.sasika_pamith()

In [ ]:
model._modules

In [ ]:
type(model._modules)

In [ ]:
model._modules.keys()

In [ ]:
model._modules['language_model']._modules.keys()

In [ ]:
model._modules['language_model']._modules['model']

In [ ]:
model._modules['language_model']._modules['lm_head']

In [ ]:
model._modules['language_model']._modules['model']._modules.keys()

In [ ]:
model._modules['language_model']._modules['model']._modules['shared']

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules.keys()

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules['embed_tokens']

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules['embed_positions']

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules['layers']

In [ ]:
!cd Florence-2-large-ft/ && du -sh *

In [ ]:
os.listdir()

In [ ]:
model._modules['language_model']._modules['model']

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules['layers']

In [ ]:
len(model._modules['language_model']._modules['model']._modules['encoder']._modules['layers'])

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules['layers'][0]._modules

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules['layers'][11]._modules

In [ ]:
model._modules['language_model']._modules['model']._modules['encoder']._modules['layernorm_embedding']

---

In [ ]:
model._modules['language_model']._modules['model']._modules['decoder']._modules['layers']

In [ ]:
model._modules['language_model']._modules['model']._modules['decoder']

In [ ]:
model.vision_tower

---
## Vision Encoder breakdown

In [ ]:
model._modules['vision_tower']

In [ ]:
model._modules['vision_tower']._modules.keys()

In [ ]:
model._modules['vision_tower']._modules['convs']

In [ ]:
model._modules['vision_tower']._modules['avgpool']

In [ ]:
len(model._modules['vision_tower']._modules['blocks'])

In [ ]:
model._modules['vision_tower']._modules['blocks'][0]

In [ ]:
model._modules['vision_tower']._modules['blocks'][1]._modules.keys()

In [ ]:
model._modules['vision_tower']._modules['blocks'][0]._modules['0']

In [ ]:
model._modules['vision_tower']._modules['blocks'][1]._modules['0']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules.keys()

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules.keys()

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['conv1']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['window_attn']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['conv2']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['ffn']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['window_attn']._modules.keys()

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['window_attn']._modules['norm']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['window_attn']._modules['fn']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['spatial_block']._modules['window_attn']._modules['drop_path']

In [ ]:
model._modules['vision_tower']._modules['blocks'][2]._modules['0']._modules['channel_block']